# **BRAIN TUMOR MULTI-CLASSIFICATION USING DL**

This project STREAMLIT app focuses on the automatic classification of brain tumors from MRI images.
I used a CNN model to identify several types of tumors (glioma, no tumor, pituitary tumor, meningioma).

# CODE

**SET UP**

In [ ]:
!git clone https://github.com/nessprjct/DataSet.git
# Define paths
TRAIN = '/content/DataSet/Brain_tumor_lowData/Training'
TEST = '/content/DataSet/Brain_tumor_lowData/Testing'

Cloning into 'DataSet'...
remote: Enumerating objects: 6014, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 6014 (delta 5), reused 0 (delta 0), pack-reused 5958 (from 2)
Receiving objects: 100% (6014/6014), 117.89 MiB | 12.16 MiB/s, done.
Resolving deltas: 100% (106/106), done.
Updating files: 100% (7209/7209), done.


In [ ]:
IMGSIZE = 64
CLASSES = ['glioma', 'NO_Tumor', 'pituitary', 'meningioma']

**IMPORT**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

**PREPROCESSING**

In [ ]:
def preprocess_images(data, IMGSIZE, CLASSES):
    images, labels = [], []
    for class_label, class_name in enumerate(CLASSES):
        class_path = os.path.join(data, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, (IMGSIZE, IMGSIZE))
                img = img / 255.0  # Normalize to [0, 1]
                images.append(img)
                labels.append(class_label)
            except Exception as e:
                print(f"Error reading {img_path}: {e}")
    return np.array(images), np.array(labels)

X_train, y_train = preprocess_images(TRAIN, IMGSIZE, CLASSES)
X_test, y_test = preprocess_images(TEST, IMGSIZE, CLASSES)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)


**CNN MODEL**

In [ ]:
# One-hot
y_train_onehot = to_categorical(y_train, num_classes=4)
y_val_onehot = to_categorical(y_val, num_classes=4)
y_test_onehot = to_categorical(y_test, num_classes=4)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMGSIZE, IMGSIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,625,668 (6.20 MB)

 Trainable params: 1,625,668 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train_onehot,
    validation_data=(X_val, y_val_onehot),
    epochs=10,
    batch_size=32,
    verbose=1
)

model.save('BTmodel.h5')
print("Model saved as 'BTmodel.h5'")


Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 240ms/step - accuracy: 0.3982 - loss: 1.3301 - val_accuracy: 0.6923 - val_loss: 0.8311
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - accuracy: 0.6400 - loss: 0.8909 - val_accuracy: 0.7769 - val_loss: 0.6248
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - accuracy: 0.7424 - loss: 0.6922 - val_accuracy: 0.7769 - val_loss: 0.5947
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 219ms/step - accuracy: 0.7877 - loss: 0.5820 - val_accuracy: 0.7923 - val_loss: 0.5669
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.8502 - loss: 0.4469 - val_accuracy: 0.8692 - val_loss: 0.5205
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.8756 - loss: 0.3625 - val_accuracy: 0.8308 - val_loss: 0.5225
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.8806 - loss: 0.3211 - val_accuracy: 0.8615 - val_loss: 0.4745
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - accuracy: 0.9216 - loss: 0.2548 - val_accuracy: 0.

Model saved as 'BTmodel.h5'


**VISUALISATION**

# STREAMLIT APP

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from keras.models import load_model
from PIL import Image

model = load_model('BTmodel.h5')
CLASSES = ['Glioma', 'No Tumor', 'Pituitary', 'Meningioma']

def preprocess_image(image):
    image = image.resize((64, 64))
    image = np.array(image) / 255.0
    return np.expand_dims(image, axis=0)

# Streamlit app
st.title("Brain Tumor Multi-Classification")
uploaded_file = st.file_uploader("Upload an MRI image", type=['jpg', 'jpeg', 'png'])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_container_width=True)
    image = preprocess_image(image)
    prediction = model.predict(image)
    predicted_class = CLASSES[np.argmax(prediction)]
    st.write(f"**Prediction:** {predicted_class}")


Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.229.109.249


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸

⠼⠴⠦⠧⠇⠏⠋
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.109.249:8501

⠙⠹⠸your url is: https://mighty-hairs-run.loca.lt
y
2025-01-11 09:55:52.655541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-11 09:55:52.687733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-11 09:55:52.698143: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 09:55:54.367539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
202